# MongoDB Database

This notebook showcases an agent designed to interact with a `MongoDB` databases. 
The agent is similar to [SQL Database](https://python.langchain.com/docs/integrations/toolkits/sql_database).

As this agent is in development, all answers may not be correct. Additionally, it is not guaranteed that the agent won't perform destructive commands on your database given certain questions.

## Initialization

In [ ]:
from langchain.agents import create_mongo_agent
from langchain.agents.agent_toolkits import MongoDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.llms.openai import OpenAI
from langchain.sql_database import MongoDatabase